In [1]:
import numpy as np
import pandas as pd

In [2]:
# !curl -Lo conda_installer.py https://raw.githubusercontent.com/deepchem/deepchem/master/scripts/colab_install.py
import conda_installer
# conda_installer.install()
conda_installer.install()
# !/root/miniconda/bin/conda info -e
import rdkit
import deepchem as dc

add /home/ali/miniconda/lib/python3.6/site-packages to PYTHONPATH
all packages are already installed


In [90]:
import os

pdb_files = ["../Datasets/PDBBindSmallSet/" + pdb for pdb in os.listdir("../Datasets/PDBBindSmallSet/") if pdb.__contains__('.pdb')]

In [ ]:
mols_dict = {}
for pdb in pdb_files:
    mols_dict.update({pdb.split('/')[-1].split('.')[0]: rdkit.Chem.MolFromPDBFile(pdb)})

In [12]:
%cd ..

from PGGCN.Featurizer import PGGCNFeaturizer

/home/ali/calstate/GBNN


In [14]:
featurizer = PGGCNFeaturizer(mols_dict, 'Datasets/PDBBindSmallSet/polished.csv',
                            "Datasets/PDBBindSmallSet/csv-files/")

In [15]:
X = featurizer.featurize()

In [87]:
import pickle

In [88]:
with open('X_smallset.pkl', 'wb') as outp:
    pickle.dump(X, outp, pickle.HIGHEST_PROTOCOL)